# tensor

In [2]:
import torch

# gradient

In [3]:
# to make all of gradient-leaf machinery clearer, let's consider this session
v1 = torch.tensor([1.0, 1.0], requires_grad=True)
v2 = torch.tensor([2.0, 2.0])

In [4]:
v_sum = v1 + v2
v_res = (v_sum * 2).sum()
v_res

tensor(12., grad_fn=<SumBackward0>)

In [5]:
v1.is_leaf, v2.is_leaf

(True, True)

In [6]:
v_sum.is_leaf, v_res.is_leaf

(False, False)

In [7]:
v1.requires_grad, v2.requires_grad

(True, False)

In [9]:
v_sum.requires_grad, v_res.requires_grad

(True, True)

In [10]:
# Let’s tell PyTorch to calculate the gradietns of our graph.
v_res.backward()
v1.grad

tensor([2., 2.])

In [11]:
v2.grad

# NN building blocks

In [12]:
# create a randomly initialized feedforward layer, with two inputs and five outputs
import torch.nn as nn 
l = nn.Linear(2, 5)
v = torch.FloatTensor([1, 2])
l(v)

tensor([ 1.0563,  1.2909, -0.2808, -0.4529, -0.7067], grad_fn=<AddBackward0>)

In [13]:
# one very convenient class that allows you to combine other layers into the pip.
s = nn.Sequential(
    nn.Linear(2, 5),
    nn.ReLU(),
    nn.Linear(5, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.Dropout(p=0.3),
    nn.Softmax(dim=1)
)
s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): Dropout(p=0.3, inplace=False)
  (6): Softmax(dim=1)
)

In [20]:
# sequential example

class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(),
            nn.Linear(5, 20),
            nn.ReLU(),
            nn.Linear(20, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        return self.pipe(x)

In [21]:
net = OurModule(num_inputs=2, num_classes=3)
print(net)

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Softmax(dim=1)
  )
)


In [22]:
v = torch.FloatTensor([[2, 3]])
out = net(v)
print(out)
print("cuda's availability is %s" % torch.cuda.is_available())
if torch.cuda.is_available():
    print("data from cuda: %s" %out.to('cuda'))

tensor([[0.5022, 0.2489, 0.2489]], grad_fn=<SoftmaxBackward0>)
cuda's availability is True
data from cuda: tensor([[0.5022, 0.2489, 0.2489]], device='cuda:0', grad_fn=<ToCopyBackward0>)


In [ ]:
# for batch_x, batch_y in iterate_batches(data, batch_size=32):       #1
#     batch_x_t = torch.tensor(batch_x)       #2
#     batch_y_t = torch.tensor(batch_y)       #3
#     out_t = net(batch_x_t)                  #4
#     loss_t = loss_function(out_t, batch_y_t)#5
#     loss_t.backward()                       #6
#     optimizer.step()                        #7
#     optimizer.zero_grad()                   #8